<a href="https://colab.research.google.com/github/SeoyeonPark-skku/SWE_2021_41_2024_2_week9/blob/main/1_Trading_strategy_using_sentiment_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas_ta vaderSentiment scikit-learn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=cb04d5d64d0a9fa5049d41af07fa4942683652ff085de35343ab3eefd0ed8bd9
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [ ]:
import yfinance as yf
import pandas as pd

# Download Apple stock data
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2024-12-31")
data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.620842,72.681289,71.373218,71.627092,135480400
2020-01-03,71.914810,72.676439,71.689950,71.847110,146322800
2020-01-06,72.487846,72.526533,70.783248,71.034709,118387200
2020-01-07,72.146950,72.753831,71.926922,72.497537,108872000
2020-01-08,73.307510,73.609745,71.849533,71.849533,132079200


In [1]:
!pip install pandas_ta



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=6f7b194f9fa4cda2a59a67d3cc1904655636445815c83a3f4672a4127c9b7d70
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [3]:
#ta is a library for technical indicators
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ec4df624a8a8863a06be0b6626f9d2c434ad14474b069487840efa57fcd6c7ab
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [10]:
#Adding techincal indicators!!

!pip install ta

import yfinance as yf
import pandas as pd
import ta


close = data[['Close']].squeeze()

rsi = ta.momentum.RSIIndicator(close=close, window=14) #Hey RSIIndicator, here's the data you need for the close prices — I'm passing it in the variable called close
data['rsi'] = rsi.rsi()   #Calling an object rsi first, then calling the method rsi

sma = ta.trend.SMAIndicator(close=close, window=20)
data['sma'] = sma.sma_indicator()


ema = ta.trend.EMAIndicator(close=close, window=20)
data['ema'] = ema.ema_indicator()


data.dropna(inplace=True)

data[['Close', 'rsi', 'sma', 'ema']].head()


Price,Close,rsi,sma,ema
Ticker,AAPL,,,
Date,,,,
2020-02-27,66.290642,22.225134,75.975916,75.071167
2020-02-28,66.251839,22.174494,75.373125,74.231231
2020-03-02,72.419945,44.012392,75.252342,74.058728
2020-03-03,70.119957,39.555297,75.026835,73.683607
2020-03-04,73.372406,47.631653,74.840760,73.653969


In [11]:
#Adding a sentiment score but for now its just random scores


import numpy as np

# Simulate daily sentiment scores from -1 to 1
np.random.seed(0)
data['sentiment'] = np.random.uniform(-1, 1, len(data)) #This line creates random values between -1 and 1 or each row(each trading day)

In [12]:
#  Creating labels(targets)

data['target'] = (data['Close'].shift(-3) > data['Close']).astype(int)
data.dropna(inplace=True)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

features = ['rsi', 'sma', 'ema', 'sentiment']
X = data[features]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False) #Do not automatically shuffle b/c time is important here

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.40      0.84      0.54        96
           1       0.64      0.18      0.28       148

    accuracy                           0.44       244
   macro avg       0.52      0.51      0.41       244
weighted avg       0.55      0.44      0.39       244

